In [19]:
magma_dir = '/home/ubuntu/magma/'
transformers_dir = '/home/ubuntu/transformers/'

## **Fine-tuning**

In [20]:
finetune_script = '"'+transformers_dir+'examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+transformers_dir+'examples/seq2seq/run_eval.py"'

### **Config**

In [6]:
import sys
sys.path.insert(0, magma_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

True

### Karger Books Para

In [7]:
model_name_or_path = 'sshleifer/bart-tiny-random'

In [8]:
data_dir = '"'+magma_dir+'datasets/karger_books_para/bart"'

output_dir = '"'+magma_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_karger_books_para_train"'

log_dir = output_dir + '/logs'

In [9]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.length_penalty = config.LENGTH_PENALTY
model_config.no_repeat_ngram_size = config.NO_REPEAT_NGRAM_SIZE

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['length_penalty'] = config.LENGTH_PENALTY
model_config.task_specific_params['summarization']['no_repeat_ngram_size'] = config.NO_REPEAT_NGRAM_SIZE
model_config_dir = '"'+magma_dir+'fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

Downloading:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

##### Fine tune

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--fp16 \
--learning_rate 3e-5 --label_smoothing 0.1 \
--sortish_sampler --freeze_embeds --adafactor \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--test_max_target_length $config.ONE_BULLET_MAX_LEN \
--do_train \
--num_train_epochs 10 \
--logging_steps 10 --save_steps 1000 --save_total_limit 3 \
--per_device_train_batch_size 4 --per_device_eval_batch_size 4 \
--do_eval --evaluation_strategy steps --eval_steps 1000 --eval_beams 2 \
--metric_for_best_model rougeL --greater_is_better True \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir

01/14/2021 11:44:08 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
01/14/2021 11:44:08 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/home/ubuntu/magma/fine-tuning/sshleifer?bart-tiny-random_karger_books_para_train', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=10.0, max_steps=-1, warmup_steps=0, logging_dir='runs/Jan14_11-44-08_ip-172-31-39-35', logging_first_step=False, logging_steps=10, save_steps=1000, save_total_limit=3, no_c

[INFO|tokenization_utils_base.py:1802] 2021-01-14 11:44:13,333 >> loading file https://huggingface.co/sshleifer/bart-tiny-random/resolve/main/vocab.json from cache at /home/ubuntu/.cache/huggingface/transformers/6c6a5c8768403bcf7d2da061e943e18fa6d570179aac8951d8df0a45e5e8071e.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
[INFO|tokenization_utils_base.py:1802] 2021-01-14 11:44:13,334 >> loading file https://huggingface.co/sshleifer/bart-tiny-random/resolve/main/merges.txt from cache at /home/ubuntu/.cache/huggingface/transformers/19b7b337f8ee4dea92f336e87e5a4e785eef3ba09323cda02617d338ec06eee0.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
[INFO|tokenization_utils_base.py:1802] 2021-01-14 11:44:13,334 >> loading file https://huggingface.co/sshleifer/bart-tiny-random/resolve/main/tokenizer.json from cache at None
[INFO|tokenization_utils_base.py:1802] 2021-01-14 11:44:13,334 >> loading file https://huggingface.co/sshleifer/bart-tiny-random/resolve/mai

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--bs 2 \
--length_penalty $config.LENAGTH_PENALTY \
--no_repeat_ngram_size $config.NO_REPEAT_NGRAM_SIZE \
--num_beams $config.NUM_BEAMS \
--dump-args

Unknown option: --
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
